In [32]:
import numpy as np 
import pandas as pd
import requests
from datetime import datetime
import matplotlib.pyplot as plt
import math
import time

In [33]:
from utils.utils import setLinkEod, setLinkIntd

In [34]:
contracts = pd.read_csv("data/portfolioTesting/c16.csv")
contracts['Start Date']= pd.to_datetime(contracts['Start Date'])
contracts=contracts.drop_duplicates(subset='Award ID').sort_values(by='Start Date')

In [35]:
response = requests.get(url=setLinkIntd("LMT", "2010-03-31", "2010-03-31", "5min"))
response1 = requests.get(url=setLinkEod("RTX", "2001-01-01", "2001-01-02"))

In [36]:
response1.json()['historical']
prices = pd.Series([entry['close'] for entry in response1.json()['historical']])        
print(prices)

0    22.19
dtype: float64


In [37]:
from dataclasses import dataclass

@dataclass
class transaction:
    contractID: int
    num: int


In [38]:
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay

def getBuyDay(contractDate, prev=3):
    uBday = CustomBusinessDay(calendar=USFederalHolidayCalendar())
    contractDate = pd.Timestamp(contractDate)
    buyDay = contractDate - prev * uBday
    return buyDay.strftime('%Y-%m-%d')


In [39]:
def meanReversion(prices, window=20, threshold=2.5):
    if len(prices) < window:
        print(f"Warning: Not enough data points for rolling window (got {len(prices)}, expected {window})")
        return False  

    sma = prices.rolling(window=window).mean()
    std_dev = prices.rolling(window=window).std()

    z_score = (prices - sma) / std_dev

    if len(z_score) > 0:
        if z_score.iloc[-1] > threshold:
            return True  
        elif z_score.iloc[-1] < -threshold:
            return True
    else:
        print("Error: 'z_score' is empty or invalid.")
    
    return False  

In [40]:
def getSellDay(contractDate, ticker):
    us_business_day = CustomBusinessDay(calendar=USFederalHolidayCalendar())
    
    contractDate = pd.Timestamp(contractDate)
    adjDate = contractDate if us_business_day.is_on_offset(contractDate) else contractDate + us_business_day
    
    startDay = adjDate - pd.Timedelta(days=30)
    endDay = adjDate - pd.Timedelta(days=1)
    
    response = requests.get(setLinkIntd(ticker, startDay.strftime('%Y-%m-%d'), endDay.strftime(('%Y-%m-%d')), "5min"))
    
    if response.status_code == 200:
        data = response.json()
        if len(data)==0: 
            try:
                response = requests.get(setLinkEod(ticker, startDay.strftime('%Y-%m-%d'), endDay.strftime(('%Y-%m-%d'))))
                data = response.json()['historical']
            except Exception as e: 
                print("Unexpected exception: "+ str(e) + "at getSellDay")
                return None
        if len(data)!= 0:
            prices = pd.Series([entry['close'] for entry in data])        
            
            if meanReversion(prices):
                return adjDate.strftime('%Y-%m-%d')  
            else:
                next_day = adjDate + us_business_day
                return getSellDay(next_day.strftime('%Y-%m-%d'), ticker)  
        else: 
            return
    else:
        print(f"Error fetching data: {response.status_code}. Returning default sell day")
        return adjDate 


In [41]:
def buy(ticker, valuation, buyDate, id): 
    global portfolio, capital
    response = requests.get(url=setLinkIntd(ticker, buyDate, buyDate, "5min"))

    if response.status_code == 200: 
        if len(response.json()) == 0: 
            response = requests.get(url=setLinkEod(ticker, buyDate, buyDate))
            try: 
                pps = response.json()['historical']['close']
            except Exception as e: 
                print("Unexpected exception: "+ str(e))
                return False
        if len(response.json())>15:
            pps = response.json()[-15]["open"]

        if (pps!=None): 
            shares = math.floor(valuation/pps)
            if ticker not in portfolio: 
                portfolio[ticker]= [transaction(contractID=id, num=shares)] 
            else: 
                portfolio[ticker].append(transaction(contractID=id, num=shares))
            capital-=shares*pps 
            print("purchased "+ str(shares) + " shares of "+ticker+" at "+ str(pps)+" on "+buyDate+". current portfolio valued at "+ str(capital))
            return True 
        else: 
            return False
    else:
        return False 

            

In [42]:
def sell(ticker, sellDate, id):
    global portfolio, capital, valuation
    response = requests.get(url=setLinkIntd(ticker, sellDate, sellDate, "5min"))
    if response.status_code == 200 and len(response.json()) >15: 
        pps = response.json()[-15]["open"]
        for t in portfolio[ticker]: 
            if t.contractID == id: 
                shares = t.num
                value = shares*pps
                capital+=value 
                portfolio[ticker].remove(t)
                valuation.append(capital)
                print("sold "+str(shares) +" shares of " + ticker + " at " + str(pps)+" on "+sellDate+". current portfolio is valued at " + str(capital))
                return True 

In [43]:
capital = 10000000
portfolio = {}
valuation = []

In [44]:
global capital
capital = 100000000
for i in range(len(contracts)): 
    if (buy(contracts.iloc[i]['Recipient Name'], capital*0.15, getBuyDay(contracts.iloc[i]['Start Date']), contracts.iloc[i]['internal_id'])):
        sell(contracts.iloc[i]['Recipient Name'], getSellDay(contracts.iloc[i]['Start Date'], ticker=contracts.iloc[i]['Recipient Name']), contracts.iloc[i]['internal_id'])
    else:
        continue

purchased 78414 shares of NOC at 191.29 on 2015-12-29. current portfolio valued at 85000185.94
sold 78414 shares of NOC at 182.7062 on 2016-01-22. current portfolio is valued at 99326909.9068
purchased 117149 shares of RTN at 127.18 on 2015-12-29. current portfolio valued at 84427900.08680001
sold 117149 shares of RTN at 124.38 on 2016-01-05. current portfolio is valued at 98998892.70680001
purchased 77629 shares of NOC at 191.29 on 2015-12-29. current portfolio valued at 84149241.29680002
sold 77629 shares of NOC at 182.7062 on 2016-01-22. current portfolio is valued at 98332540.89660002
purchased 73131 shares of MCK at 201.69 on 2015-12-29. current portfolio valued at 83582749.50660002
sold 73131 shares of MCK at 195 on 2016-01-04. current portfolio is valued at 97843294.50660002
Unexpected exception: list indices must be integers or slices, not str
purchased 115399 shares of RTN at 127.18 on 2015-12-29. current portfolio valued at 83166849.68660003
sold 115399 shares of RTN at 124.3

KeyboardInterrupt: 